In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
def get_wiki_page(params):
    return requests.get('https://en.wikipedia.org/w/api.php', params=params)

In [5]:
def get_NCAA_D1_teams():
    params = {
        'action': 'parse',
        'page': "List of NCAA Division I institutions",
        'format': 'json',
        'prop': 'text',
    }
    res = get_wiki_page(params)
    if res.status_code == 200:
        page = res.json()['parse']['text']['*']
        soup = BeautifulSoup(page, 'html.parser')
        table = soup.find_all('table')[1]
        team_names = []
        for row in table.find_all('tr')[2:]:
            cells = row.find_all('td')
            if len(cells) < 3:
                continue
            nickname = cells[2].find('a')['title']
            team_names.append(nickname)
    return team_names

teams = get_NCAA_D1_teams()

In [6]:
def get_team_data(team_name):
    params = {
        'action': 'query',
        'titles': team_name,
        'format': 'json',
        'prop': 'extracts',
        'explaintext': 'true',
    }
    res = get_wiki_page(params)
    pages = res.json()['query']['pages']
    if pages:
        return pages[next(iter(pages))]['extract']
    

In [7]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.environ.get("GOOGLE_CLOUD_API_KEY")

# Initialize the Gemini API
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(
    "gemini-1.5-pro-latest",
    generation_config=genai.GenerationConfig(
        max_output_tokens=8000,
        temperature=0,
    ),
)

def generate_embeddings_from_list(text_list: list):
    result = genai.embed_content(model="models/text-embedding-004", content=text_list, task_type="question_answering")
    for embedding in result['embedding']:
        print(str(embedding)[:50], '... TRIMMED]')
    return result


def generate_embeddings(text: str):
    result = genai.embed_content(model="models/text-embedding-004", content=text, task_type="retrieval_document")
    # Print just a part of the embedding to keep the output manageable
    print(str(result['embedding'])[:50], '... TRIMMED]')
    return result

In [27]:
team1 = get_team_data(teams[0])
team1_embed_name = generate_embeddings(teams[0])

[0.0170326, 0.020390132, -0.026702601, 0.046677213 ... TRIMMED]


In [25]:
import re
sentences = []
def format_data(data):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', data)
    sentences = list(filter(None, map(str.strip, sentences)))

    for sentence in sentences:
        print(sentence)
    return sentences

sentences = format_data(team1)

Abilene Christian Wildcats (variously ACU or ACU Wildcats) refers to the sports teams of Abilene Christian University located in Abilene, Texas.
The Wildcats joined the Western Athletic Conference (WAC) on July 1, 2021, after having spent the previous eight years in the Southland Conference.
The nickname "Wildcat" is derived from the mascot of the team.
== Sports sponsored ==


== History ==
A member of the Western Athletic Conference, Abilene Christian sponsors teams in seven men's and eight women's NCAA sanctioned sports.
On December 6, 1923, Abilene Christian applied for admission to Texas Intercollegiate Athletic Association during the conference annual meeting in Dallas.
The Wildcats had been considering joining the conference for several years but funding for the athletic department preventing them to join the conference sooner.
On July 1, 2013, Abilene Christian returned to the Southland Conference as one of four new members.
The university, a charter member of the Southland Con

In [14]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.environ.get("GOOGLE_CLOUD_API_KEY")

# Initialize the Gemini API
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(
    "gemini-1.5-pro-latest",
    generation_config=genai.GenerationConfig(
        max_output_tokens=8000,
        temperature=0,
    ),
)

def generate_embeddings_from_list(text_list: list):
    result = genai.embed_content(model="models/text-embedding-004", content=text_list, task_type="question_answering")
    for embedding in result['embedding']:
        print(str(embedding)[:50], '... TRIMMED]')
    return result


def generate_embeddings(text: str): # dim: 768
    result = genai.embed_content(model="models/text-embedding-004", content=text, task_type="retrieval_document")
    # Print just a part of the embedding to keep the output manageable
    print(str(result['embedding'])[:50], '... TRIMMED]')
    return result

In [46]:
from pymilvus import MilvusClient, DataType
import os
import json
from dotenv import load_dotenv

load_dotenv()


class MilvusApp:
    def __init__(self, api_key, api_endpoint):
        self.api_key = api_key
        self.api_endpoint = api_endpoint
        self.client = self.connect()

    def connect(self):
        self.client = MilvusClient(uri=self.api_endpoint, token=self.api_key) 
        if not self.client:
            raise Exception("Client not established.")

    def disconnect(self):
        if self.client:
            self.client.close()


    def create_embedding_schema(self):
        # each entry is one article
        schema = MilvusClient.create_schema(
            auto_id=True,
            enable_dynamic_field=True
        )

        schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
        schema.add_field(field_name="team_name_vector", datatype=DataType.FLOAT_VECTOR, dim=768)
        schema.add_field(field_name="article_json", datatype=DataType.JSON)
        return schema
    

    def format_data(self, team_name, team_name_vector, article_text_list):
        data = {
            "team_name_vector": team_name_vector,
            "article_json": {
                'team_name': team_name,
                'article_content': article_text_list, 
            }  
        }
        return data
    

    def upload_data(self, collection_name, data):
        res = self.client.insert(
            collection_name=collection_name,
            data=data
        )
        print(res)


    def create_collection(self, collection_name, schema=None):
        res = self.client.list_collections()
        print(res)
        if collection_name in res:
            raise Exception(f"'{collection_name}' already exists in cluster.\n{self.client.describe_collection(collection_name=collection_name)}")
        
        index_params = self.client.prepare_index_params()

        # index_params.add_index(
        #     field_name="id",
        #     index_type="STL_SORT"
        # )

        index_params.add_index(
            field_name="team_name_vector", 
            index_type="AUTOINDEX", # zilliz
            metric_type="L2",
            params={}
        )
        
        if not schema:
            schema = self.create_embedding_schema()
        print(index_params)
        print(schema)
        collection = self.client.create_collection(
                collection_name=collection_name, 
                schema=schema, 
                index_params=index_params)
        return collection


    def load_collection(self, collection_name):
        # for searching entities purpose only 
        self.client.load_collection(
            collection_name=collection_name
        )
        res = self.client.get_load_state(
            collection_name=collection_name
        )
        print(res)
    

    def drop_collection(self, collection_name):
        self.client.drop_collection(
            collection_name=collection_name
        )


    def upload_data(self, collection_name, data):
        res = self.client.insert(
            collection_name=collection_name,
            data=data
        )
        print(res)
    

    def search_data(self, collection_name, query_embedding, top_k=5):
        self.load_collection(collection_name)
        # Single vector search
        res = self.client.search(
            collection_name=collection_name,
            data=query_embedding,
            limit=top_k, # Max. number of search results to return
            search_params={"metric_type": "L2"}
        )

        result = json.dumps(res, indent=4)
        print(result)
    

MILVUS_API_KEY = os.environ.get('MILVUS_API_KEY')
MILVUS_API_ENDPOINT = "https://in03-df5c1c6a3c98f62.api.gcp-us-west1.zillizcloud.com"
milvus_app = MilvusApp(api_key=MILVUS_API_KEY, api_endpoint=MILVUS_API_ENDPOINT)
milvus_app.connect()

collection_name = "Pactrivia_data"
milvus_app.create_collection(collection_name)

['medium_articles', 'Pactrivia_data']


Exception: 'Pactrivia_data' already exists in cluster.
{'collection_name': 'Pactrivia_data', 'auto_id': True, 'num_shards': 1, 'description': '', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'team_name_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}, {'field_id': 102, 'name': 'article_json', 'description': '', 'type': <DataType.JSON: 23>, 'params': {}}], 'aliases': [], 'collection_id': 449686286190462718, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True}

In [47]:
data = milvus_app.format_data(teams[0], team1_embed_name['embedding'], sentences)
data

{'team_name_vector': [0.0170326,
  0.020390132,
  -0.026702601,
  0.046677213,
  0.039016515,
  0.007283795,
  0.009143837,
  0.029600112,
  -0.026886836,
  0.025534192,
  0.025696287,
  -0.007201744,
  0.012390414,
  0.009116536,
  -0.004104345,
  -0.0038682777,
  0.024002297,
  0.042020816,
  -0.116137594,
  -0.012162356,
  -0.0050529717,
  -0.044144507,
  0.03629511,
  0.015217971,
  -0.015468548,
  -0.03336388,
  0.024729151,
  -0.04968274,
  -0.036115717,
  -0.003124638,
  0.017102432,
  0.061775114,
  0.035321817,
  -0.07389817,
  -0.021242108,
  0.051807176,
  0.0006799167,
  -0.00852795,
  0.09410862,
  -0.047147274,
  -0.076967046,
  0.04205411,
  -0.038590286,
  0.014241451,
  0.0065736067,
  -0.023374798,
  -0.013462055,
  0.0382384,
  -0.05141897,
  0.04079918,
  -0.010313023,
  0.087015696,
  -0.06790516,
  0.024281228,
  -0.01561109,
  -0.01944819,
  -0.03475458,
  -0.06838631,
  0.0062594297,
  -0.03233706,
  0.023643918,
  0.0037873492,
  0.03271537,
  0.0041422443,
  0

In [48]:
milvus_app.upload_data("Pactrivia_data", data)  

{'insert_count': 1, 'ids': [449686286190255884]}


In [53]:
milvus_app.search_data("Pactrivia_data", team1_embed_name['embedding'], 1)

RPC error: [search], <ParamError: (code=1, message=`search_data` value [0.0170326, 0.020390132, -0.026702601, 0.046677213, 0.039016515, 0.007283795, 0.009143837, 0.029600112, -0.026886836, 0.025534192, 0.025696287, -0.007201744, 0.012390414, 0.009116536, -0.004104345, -0.0038682777, 0.024002297, 0.042020816, -0.116137594, -0.012162356, -0.0050529717, -0.044144507, 0.03629511, 0.015217971, -0.015468548, -0.03336388, 0.024729151, -0.04968274, -0.036115717, -0.003124638, 0.017102432, 0.061775114, 0.035321817, -0.07389817, -0.021242108, 0.051807176, 0.0006799167, -0.00852795, 0.09410862, -0.047147274, -0.076967046, 0.04205411, -0.038590286, 0.014241451, 0.0065736067, -0.023374798, -0.013462055, 0.0382384, -0.05141897, 0.04079918, -0.010313023, 0.087015696, -0.06790516, 0.024281228, -0.01561109, -0.01944819, -0.03475458, -0.06838631, 0.0062594297, -0.03233706, 0.023643918, 0.0037873492, 0.03271537, 0.0041422443, 0.008258641, -0.026560662, -0.009204249, -0.03556133, -0.07665897, 0.013492097,

{'state': <LoadState: Loaded>}


ParamError: <ParamError: (code=1, message=`search_data` value [0.0170326, 0.020390132, -0.026702601, 0.046677213, 0.039016515, 0.007283795, 0.009143837, 0.029600112, -0.026886836, 0.025534192, 0.025696287, -0.007201744, 0.012390414, 0.009116536, -0.004104345, -0.0038682777, 0.024002297, 0.042020816, -0.116137594, -0.012162356, -0.0050529717, -0.044144507, 0.03629511, 0.015217971, -0.015468548, -0.03336388, 0.024729151, -0.04968274, -0.036115717, -0.003124638, 0.017102432, 0.061775114, 0.035321817, -0.07389817, -0.021242108, 0.051807176, 0.0006799167, -0.00852795, 0.09410862, -0.047147274, -0.076967046, 0.04205411, -0.038590286, 0.014241451, 0.0065736067, -0.023374798, -0.013462055, 0.0382384, -0.05141897, 0.04079918, -0.010313023, 0.087015696, -0.06790516, 0.024281228, -0.01561109, -0.01944819, -0.03475458, -0.06838631, 0.0062594297, -0.03233706, 0.023643918, 0.0037873492, 0.03271537, 0.0041422443, 0.008258641, -0.026560662, -0.009204249, -0.03556133, -0.07665897, 0.013492097, 0.009576

In [ ]:
milvus_app.disconnect()